# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [73]:
from pymongo import MongoClient
import pandas as pd
import time

client = MongoClient("localhost:27017")
client

db = client["Ironhack"]
c = db.get_collection("companies")

In [227]:
c.find_one({}, skip=7)

{'_id': ObjectId('52cdef7c4bab8bd675297d91'),
 'name': 'Geni',
 'permalink': 'geni',
 'crunchbase_url': 'http://www.crunchbase.com/company/geni',
 'homepage_url': 'http://www.geni.com',
 'blog_url': 'http://blog.geni.com',
 'blog_feed_url': 'http://blog.geni.com/index.rdf',
 'twitter_username': 'geni',
 'category_code': 'web',
 'number_of_employees': 18,
 'founded_year': 2006,
 'founded_month': 6,
 'founded_day': 1,
 'deadpooled_year': None,
 'deadpooled_month': None,
 'deadpooled_day': None,
 'deadpooled_url': None,
 'tag_list': 'geni, geneology, social, family, genealogy',
 'alias_list': '',
 'email_address': '',
 'phone_number': '',
 'description': 'Geneology social network site',
 'created_at': 'Thu May 31 19:52:34 UTC 2007',
 'updated_at': 'Wed Oct 10 14:01:29 UTC 2012',
 'overview': '<p>Geni is an online community of casual and expert genealogists working together to break the barriers limiting family history research today.</p>\n\n<p>They share knowledge and collaborate to creat

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [13]:
list(c.find({"name":"Babelgum"},  projection = {"_id":0, "name":1, "outcode":1}))[0]

{'name': 'Babelgum'}

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [59]:
filter_1 = {}
projection = {"name": 1, "number_of_employees": 1, "_id": 0}
results = list(c.find(filter_1, projection).sort("number_of_employees", -1).limit(20))
pd.DataFrame(results)

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [60]:
filter_2 = {"founded_year": {"$gte": 2000, "$lte": 2005}}
projection = {"name": 1, "founded_year": 1, "_id": 0}

results2 = list(c.find(filter_2, projection).sort("founded_year", -1))
pd.DataFrame(results2)

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Omnidrive,2005
3,Helio,2005
4,Jingle Networks,2005
...,...,...
3729,Vigilos,2000
3730,Block Shield,2000
3731,Netrake,2000
3732,EnterSys Group,2000


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [98]:
filter_3 = {"ipo.valuation_amount" :{"$gt": 100000000}, "founded_year": {"$lte": 2010}}
projection = {"name": 1, "ipo.valuation_amount": 1, "_id": 0}

results3 = list(c.find(filter_3, projection).sort("ipo.valuation_amount", -1))
pd.DataFrame(results3)

,name,ipo
0,GREE,{'valuation_amount': 108960000000.0}
1,Facebook,{'valuation_amount': 104000000000.0}
2,Amazon,{'valuation_amount': 100000000000.0}
3,Twitter,{'valuation_amount': 18100000000.0}
4,Groupon,{'valuation_amount': 12800000000.0}
5,Tencent,{'valuation_amount': 11000000000.0}
6,Western Digital,{'valuation_amount': 9430000000.0}
7,LinkedIn,{'valuation_amount': 9310000000.0}
8,BMC Software,{'valuation_amount': 6000000000.0}
9,Rackspace,{'valuation_amount': 5440000000.0}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [102]:
filter_4 = {"number_of_employees" :{"$lte": 1000}, "founded_year": {"$lte": 2005}}
projection = {"name": 1, "founded_year": 1, "number_of_employees":1, "_id": 0}

results4 = list(c.find(filter_4, projection).sort("number_of_employees", -1).limit(10))
pd.DataFrame(results4)

,name,number_of_employees,founded_year
0,Sonus Networks,1000,1997
1,SK Net Service Company Ltd,1000,2001
2,Gumtree,1000,2005
3,Akamai Technologies,1000,1998
4,Omniture,1000,1996
5,Antal International,1000,1993
6,OC'LIANE,1000,2003
7,FANUC Robotics America,1000,1982
8,Telenav,1000,1999
9,Yodle,1000,2005


### 6. All the companies that don't include the `partners` field.

In [123]:
filter_4 = {"companies" : {"$ne": "partners"}}
projection = {"name": 1, "_id": 0, "partners":1}

results4 = list(c.find(filter_4, projection).sort("name", -1).limit(10))
pd.DataFrame(results4)

,name,partners
0,zyntroPICS,[]
1,zweitgeist,[]
2,zip2,[]
3,zip realty,[]
4,zintin,[]
5,zimtkorn,[]
6,zembly,[]
7,zebraspot design,[]
8,zeaLOG,[]
9,zappn,[]


### 7. All the companies that have a null type of value on the `category_code` field.

In [127]:
filter_5 = {"category_code" : {"$type": "null"}}
projection = {"name": 1, "_id": 0, "category_code":1}

results5 = list(c.find(filter_5, projection).sort("companies", -1).limit(10))
pd.DataFrame(results5)

,name,category_code
0,MMDAYS,None
1,Inside Group,None
2,VidKing,None
3,Touch Clarity,None
4,Drigg,None
5,Level9 Media,None
6,Snimmer,None
7,Collective,None
8,KoolIM,None
9,SpaceTime,None


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [129]:
filter_6 = {"number_of_employees" :{"$lt": 1000, "$gte":100}}
projection = {"name": 1, "number_of_employees":1, "_id": 0}

results6 = list(c.find(filter_6, projection).sort("number_of_employees", -1).limit(10))
pd.DataFrame(results6)

,name,number_of_employees
0,Datamonitor,984
1,Infinera Corporation,974
2,Box,950
3,NorthPoint Communications Group,948
4,888 Holdings,931
5,Forrester Research,903
6,SonicWALL,900
7,Relax Solutions Pvt Ltd,900
8,Webmetrics,900
9,InMobi,900


### 9. Order all the companies by their IPO price in a descending order.

In [134]:
filter_7 = {}
projection = {"name": 1, "ipo.valuation_amount":1, "_id": 0}

results7 = list(c.find(filter_7, projection).sort("ipo.valuation_amount", -1).limit(10))
pd.DataFrame(results7)

,name,ipo
0,GREE,{'valuation_amount': 108960000000.0}
1,Facebook,{'valuation_amount': 104000000000.0}
2,Amazon,{'valuation_amount': 100000000000.0}
3,Twitter,{'valuation_amount': 18100000000.0}
4,Groupon,{'valuation_amount': 12800000000.0}
5,Tencent,{'valuation_amount': 11000000000.0}
6,Western Digital,{'valuation_amount': 9430000000.0}
7,LinkedIn,{'valuation_amount': 9310000000.0}
8,BMC Software,{'valuation_amount': 6000000000.0}
9,Rackspace,{'valuation_amount': 5440000000.0}


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [140]:
filter_8 = {}
projection = {"name": 1, "number_of_employees":1, "_id": 0}

results8 = list(c.find(filter_8, projection).sort("number_of_employees", -1).limit(10))
pd.DataFrame(results8)

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [146]:
filter_9 = {"founded_month" :{"$gt": 6}}
projection = {"name": 1, "founded_month":1, "_id": 0}

results9 = list(c.find(filter_9, projection).sort("founded_month", 1).limit(1000))
pd.DataFrame(results9)

,name,founded_month
0,Buxfer,7
1,Klatcher,7
2,Stanfy,7
3,Pando Networks,7
4,Inuda Innovations,7
...,...,...
995,SDH Interactive,8
996,iFlaker,8
997,Jobitorial,8
998,ProjectX Technology,8


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [149]:
filter_10 = {"founded_year" :{"$lt": 2000}, "acquisition.price_amount": {"$gte":10000000}}
projection = {"name": 1, "acquisition.price_amount":1, "_id": 0}

results10 = list(c.find(filter_10, projection).sort("acquisition.price_amount", -1).limit(10))
pd.DataFrame(results10)

,name,acquisition
0,BEA Systems,{'price_amount': 8500000000.0}
1,Navteq,{'price_amount': 8100000000.0}
2,Sun Microsystems,{'price_amount': 7400000000.0}
3,Pixar,{'price_amount': 7400000000.0}
4,LSI,{'price_amount': 6600000000.0}
5,National Semiconductor,{'price_amount': 6500000000.0}
6,aQuantive,{'price_amount': 6400000000.0}
7,Siebel Systems,{'price_amount': 5850000000.0}
8,Sybase,{'price_amount': 5800000000.0}
9,Affiliated Computer Services,{'price_amount': 5750000000.0}


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [158]:
filter_11 = {"acquisition.acquired_year" :{"$gte": 2010}}
projection = {"name": 1, "acquired_year":1, "acquisition.price_amount":1, "_id": 0}

results11 = list(c.find(filter_11, projection).sort("acquisition.price_amount", -1))
pd.DataFrame(results11)

,name,acquisition
0,T-Mobile,{'price_amount': 39000000000.0}
1,Goodrich Corporation,{'price_amount': 18400000000.0}
2,LSI,{'price_amount': 6600000000.0}
3,National Semiconductor,{'price_amount': 6500000000.0}
4,Sybase,{'price_amount': 5800000000.0}
...,...,...
1095,ALOT,{'price_amount': None}
1096,Nellix,{'price_amount': None}
1097,Celestial Semiconductor,{'price_amount': None}
1098,MyChances,{'price_amount': None}


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [160]:
filter_12 = {}
projection = {"name": 1, "founded_year":1, "_id": 0}

results12 = list(c.find(filter_12, projection).sort("founded_year", -1).limit(10))
pd.DataFrame(results12)

,name,founded_year
0,Wamba,2013
1,Gimigo,2013
2,Clowdy,2013
3,Fixya,2013
4,Fluc,2013
5,SEOGroup,2013
6,Pikk,2013
7,WhosCall,2013
8,iBazar,2013
9,Advaliant,2013


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [165]:
filter_13 = {"founded_day" :{"$lte": 7}}
projection = {"name": 1, "founded_day":1, "_id": 0}

results13 = list(c.find(filter_13, projection).sort("founded_day", -1))
pd.DataFrame(results13)

,name,founded_day
0,Meetup,7
1,Google,7
2,Jive Software,7
3,Rakuten,7
4,Driftr,7
...,...,...
3216,Gambolio,1
3217,SEO Sumo,1
3218,Kicker Films,1
3219,Getyoo,1


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [175]:
filter_14 = {"category_code" :"web", "number_of_employees": {"$gte":4000}}
projection = {"name": 1, "category_code":1, "_id": 0, "number_of_employees":1 }

results14 = list(c.find(filter_14, projection).sort("number_of_employees", 1))
pd.DataFrame(results14)

,name,category_code,number_of_employees
0,Expedia,web,4400
1,AOL,web,8000
2,Webkinz,web,8657
3,Rakuten,web,10000
4,Los Angeles Times Media Group,web,10000
5,Groupon,web,10000
6,Yahoo!,web,13600
7,eBay,web,15000
8,Experian,web,15500


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [189]:
filter_15 = {"acquisition.price_currency_code" :"EUR", "acquisition.price_amount": {"$gte":10000000}}
projection = {"name": 1, "acquisition.price_amount":1, "_id": 0, "acquisition.price_currency_code":1 }

results15 = list(c.find(filter_15, projection).sort("acquisition.price_amount", -1))
pd.DataFrame(results15)

,name,acquisition
0,Apertio,"{'price_amount': 140000000, 'price_currency_co..."
1,Webedia,"{'price_amount': 70000000, 'price_currency_cod..."
2,Tuenti Technologies,"{'price_amount': 70000000, 'price_currency_cod..."
3,BioMed Central,"{'price_amount': 43400000, 'price_currency_cod..."
4,Greenfield Online,"{'price_amount': 40000000, 'price_currency_cod..."
5,ZYB,"{'price_amount': 31500000, 'price_currency_cod..."
6,Wayfinder,"{'price_amount': 24000000, 'price_currency_cod..."


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [197]:
filter_16 = {"acquisition.acquired_month" :{"$lte": 3}}
projection = {"name": 1, "acquisition":1, "_id": 0}

results16 = list(c.find(filter_16, projection).sort("acquisition", -1).limit(10))
pd.DataFrame(results16)

,name,acquisition
0,T-Mobile,"{'price_amount': 39000000000.0, 'price_currenc..."
1,BEA Systems,"{'price_amount': 8500000000.0, 'price_currency..."
2,Pixar,"{'price_amount': 7400000000.0, 'price_currency..."
3,Puget Energy,"{'price_amount': 7400000000.0, 'price_currency..."
4,Siebel Systems,"{'price_amount': 5850000000.0, 'price_currency..."
5,Sterling Commerce,"{'price_amount': 3900000000.0, 'price_currency..."
6,SuccessFactors,"{'price_amount': 3400000000.0, 'price_currency..."
7,WebEx,"{'price_amount': 2900000000.0, 'price_currency..."
8,WebEx,"{'price_amount': 2900000000.0, 'price_currency..."
9,GeoCities,"{'price_amount': 2870000000.0, 'price_currency..."


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [208]:
filter_17 = {"founded_year" :{"$gte": 2000, "$lte": 2010}, "acquisition.acquired_year":{"$lt": 2011}}
projection = {"name": 1, "founded_year":1, "acquisition.acquired_year":1, "_id": 0}

results17 = list(c.find(filter_17, projection).sort("founded_year", -1))
pd.DataFrame(results17)

,name,founded_year,acquisition
0,Vusion,2009,{'acquired_year': 2009}
1,Digimix,2009,{'acquired_year': 2009}
2,About Me!,2009,{'acquired_year': 2007}
3,Divvyshot,2009,{'acquired_year': 2010}
4,Cloudkick,2009,{'acquired_year': 2010}
...,...,...,...
708,Skywire Software,2000,{'acquired_year': 2008}
709,Sirific Wireless,2000,{'acquired_year': 2009}
710,LogicLibrary,2000,{'acquired_year': 2008}
711,EnterSys Group,2000,{'acquired_year': 2009}


### 20. All the companies that have been 'deadpooled' after the third year.

In [226]:
filter_18 = {"deadpooled_year": {"$gt": {"$sum": ["founded_year", 3]}}}
projection = {"name": 1, "founded_year":1, "deadpooled_year":1, "_id": 0}

results18  = list(c.find(filter_18, projection).sort("founded_year", -1))
pd.DataFrame(results18)

""


**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```